worker ids:
46994197,CA,Redondo Beach, Salim, 21
47200615,NY,Ossining, Hee Dong, 21, 5
46545976,NY,Newburgh, Eva-Maria, 21, 3
46092070,NY,New City, Sara, 18, 2

In [ ]:
import pandas as pd
from datetime import datetime

name = {46994197:"Salim", 47200615:"Hee Dong", 46545976: "Eva-Maria", 46092070: "Sara"}
file_name = "/dccstor/srosent2/generative/appen/job_1988758NQLFQA.json"
data = pd.read_json(file_name, lines=True)

In [ ]:

skip_workers = {}
time_workers = {}
num_complete = {}

for worker in name:
    skip_workers[worker] = 0
    num_complete[worker] = 0
    # time_workers[worker] = 0

for index, row in data.iterrows():
    info = row['results']['judgments'][0]
    # print(row['results'])
        
    start = datetime.strptime(info['started_at'][11:-6],"%H:%M:%S")
    end = datetime.strptime(info['created_at'][11:-6],"%H:%M:%S")
    if info['worker_id'] not in time_workers:
        time_workers[info['worker_id']] = end-start
    else:
        time_workers[info['worker_id']] += end-start
    num_complete[info['worker_id']] += 1
    print(info['unit_data']['question_id']  + ": " + info['unit_data']['question'])
    print(name[info['worker_id']] + ": " + str(end-start))
    print("PARAGRAPH:")
    for sentence in info['unit_data']['long_answer']:
        print(sentence)
    if 'minimal_answer' in info['unit_data'] and info['unit_data']['minimal_answer'] is not None:
        print("minimal answer: " + info['unit_data']['minimal_answer'])
    if 'skipthere_is_no_answer' in info['data'] or 'there_is_no_answer' in info['data']:
        skip_workers[info['worker_id']] += 1
        print("SKIP")
    else:
        print("ANSWER: ")
        if 'paragraph_sentences' in info['data']:
            for sentence in info['data']['paragraph_sentences']:
                print(sentence)
        else:
            print("no sentences selected")
            print(info['data'])
        if 'type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_' not in info['data']:
        #    print(info['data']['type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_'])
        #else:
            print("no answer provided")
   
    print("---")
    # break
    # print(str(info['worker_id']) + "," + info['region'] + "," + info['city'] + "," + info['started_at'])
print(skip_workers)
for worker in name:
    print(name[worker] + ": " + str(time_workers[worker]/num_complete[worker]) + " skip: " +  str(skip_workers[worker]))

In [1]:
import pandas as pd
from datetime import datetime

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 46092070: "Sara", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie", 46954475: "Arafat"}
file_name = "/dccstor/srosent2/generative/appen/job_2004889.json"
data = pd.read_json(file_name, lines=True)

In [20]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rougeL(answer1, answer2):
    scores = scorer.score(answer1, answer2)
    return scores

def process_data(data):

    avg_rouge_all =  {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []} 
    partial =  {"Hee Dong": 0, "Eva Maria": 0, "Sara": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Arafat": 0} 
    
    for index, row in data.iterrows():
        avg_rouge = {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []}
        print(str(row['data']['question_id']) + ": " + row['data']['question'])
        print("Title: " + str(row['data']['title']))
        for sentence in row['data']['long_answer']:
            print(sentence)
        print("---")
        judgements = row['results']['judgments']
        skip = set()
        scores = {}

        i = 0
        for i in range(len(judgements)):
            answer_type = judgements[i]['data']['how_would_you_describe_the_questionanswer']

            if answer_type == 'complete' or answer_type == 'partial':
                if answer_type == 'partial':
                    partial[name[judgements[i]['worker_id']]] += 1
                answer1 = judgements[i]['data']['type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_']
                print(name[judgements[i]['worker_id']] + ": " + answer1 + " (" + answer_type + ")")
                if 'comments_optional' in judgements[i]['data']:
                    print("comment: " + judgements[i]['data']['comments_optional'])
            else:
                skip.add(name[judgements[i]['worker_id']] + ": " + answer_type)
                continue

            for j in range(i+1,len(judgements)):
                answer_type = judgements[j]['data']['how_would_you_describe_the_questionanswer']

                if answer_type == 'complete' or answer_type == 'partial':
                    answer2 = judgements[j]['data']['type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_']
                    # print(str(judgements[j]['worker_id']) + ": " + answer)
                else:
                    continue
                score = rougeL(answer1, answer2)
                scores[name[judgements[i]['worker_id']] + "-" + name[judgements[j]['worker_id']]] = score['rougeL'].fmeasure
                avg_rouge[name[judgements[i]['worker_id']]].append(score['rougeL'].fmeasure)
                avg_rouge[name[judgements[j]['worker_id']]].append(score['rougeL'].fmeasure)
        print("skip: " + str(skip))
        scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

        for score in scores:
            print(score)

        print("Average: ")
        for worker in avg_rouge:
            if len(avg_rouge[worker]) > 0:
                avg = sum(avg_rouge[worker])/len(avg_rouge[worker])
                print(worker + ": " + str(avg))
                avg_rouge_all[worker].append(avg)
        print("-----------------")
    for worker in avg_rouge_all:
        avg = sum(avg_rouge_all[worker])/len(avg_rouge_all[worker])
        print("avg rouge: " + worker + ": " + str(avg) + " (# answered: " + str(len(avg_rouge_all[worker])) + ")")
        print("partial count: " + worker + ": " + str(partial[worker]))
        print("-----------------")

process_data(data)

-7210339619807281790: what kind of bird is black and white with red on its chest
Title: Rose-breasted grosbeak
The adult male in breeding plumage has a black head , wings , back , and tail , and a bright rose - red patch on its breast ; the wings have two white patches and rose - red linings .
Its underside and rump are white .
Males in nonbreeding plumage have largely white underparts , supercilium , and cheeks .
The upperside feathers have brown fringes , and most wing feathers white ones , giving a scaly appearance .
The bases of the primary remiges are also white .
The coloration renders the adult male rose - breasted grosbeak ( even while wintering ) unmistakable if seen well .
---
Eva Maria: The coloration renders the adult male rose - breasted grosbeak ( even while wintering ) unmistakable if seen well.
Chie: The adult male in breeding plumage has a black head , wings , back , and tail , and a bright rose - red patch on its breast ; the wings have two white patches and rose - re

In [2]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
print(scores)

{'rouge1': Score(precision=0.75, recall=0.6666666666666666, fmeasure=0.7058823529411765), 'rougeL': Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)}
